# Ontology-Driven Clinical Decision Support - End-to-End Experiments

This notebook provides a comprehensive end-to-end experimentation pipeline for the Lung Cancer Assistant (LCA) system.

## Experiments Covered

1. **Environment Setup & Validation**
2. **Ontology Construction & Inspection** (LUCADA + SNOMED-CT)
3. **Guideline Rules Engine** (NICE CG121)
4. **Individual Agent Testing** (all 6 core agents)
5. **Full 6-Agent Workflow** (LangGraph pipeline)
6. **Specialized Agents** (Biomarker, NSCLC, Comorbidity)
7. **Multi-Patient Cohort Processing**
8. **Synthetic Data Generation & Batch Analysis**
9. **Analytics** (Survival, Uncertainty, Counterfactual)
10. **Visualization & Reporting**

---
## 1. Environment Setup & Validation

In [ ]:
import sys
import os
import json
import time
import warnings
warnings.filterwarnings('ignore')

# Ensure project root is on the path
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '.'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")
print(f"Python version: {sys.version}")

In [ ]:
# Validate core dependencies
dependencies = {
    'pydantic': 'pydantic',
    'owlready2': 'owlready2',
    'rdflib': 'rdflib',
    'numpy': 'numpy',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
}

optional_deps = {
    'langchain': 'langchain',
    'langgraph': 'langgraph',
    'neo4j': 'neo4j',
    'lifelines': 'lifelines',
    'sentence_transformers': 'sentence-transformers',
}

print("=== Core Dependencies ===")
for name, pkg in dependencies.items():
    try:
        mod = __import__(name)
        ver = getattr(mod, '__version__', 'installed')
        print(f"  [OK] {pkg}: {ver}")
    except ImportError:
        print(f"  [MISSING] {pkg} - install with: pip install {pkg}")

print("\n=== Optional Dependencies ===")
for name, pkg in optional_deps.items():
    try:
        mod = __import__(name)
        ver = getattr(mod, '__version__', 'installed')
        print(f"  [OK] {pkg}: {ver}")
    except ImportError:
        print(f"  [SKIP] {pkg} - not installed (some experiments will use fallbacks)")

In [ ]:
# Common imports used throughout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from pprint import pprint

# Plotting config
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
sns.set_theme(style='whitegrid')

print("Common imports loaded.")

---
## 2. Ontology Construction & Inspection

Build the LUCADA OWL 2 ontology and inspect its structure.

In [ ]:
from backend.src.ontology.lucada_ontology import LUCADAOntology
from backend.src.ontology.snomed_loader import SNOMEDLoader

# Create the ontology
lucada = LUCADAOntology()
onto = lucada.create()

print(f"Ontology IRI: {onto.base_iri}")
print(f"Number of classes: {len(list(onto.classes()))}")
print(f"Number of object properties: {len(list(onto.object_properties()))}")
print(f"Number of data properties: {len(list(onto.data_properties()))}")

In [ ]:
# List all ontology classes grouped by hierarchy
print("=== LUCADA Ontology Classes ===")
for cls in sorted(onto.classes(), key=lambda c: c.name):
    parents = [p.name for p in cls.is_a if hasattr(p, 'name')]
    print(f"  {cls.name} -> subclass of: {parents}")

In [ ]:
# Inspect SNOMED-CT mappings
snomed = SNOMEDLoader()
print("=== SNOMED-CT Lung Cancer Codes ===")
for concept_name, code in sorted(snomed.LUNG_CANCER_CODES.items()):
    print(f"  {concept_name}: {code}")

print(f"\nTotal SNOMED codes available: {len(snomed.LUNG_CANCER_CODES)}")

In [ ]:
# Visualize ontology class hierarchy
class_names = [cls.name for cls in onto.classes()]
parent_counts = {}
for cls in onto.classes():
    for parent in cls.is_a:
        if hasattr(parent, 'name'):
            parent_counts[parent.name] = parent_counts.get(parent.name, 0) + 1

if parent_counts:
    fig, ax = plt.subplots(figsize=(14, 6))
    top_parents = dict(sorted(parent_counts.items(), key=lambda x: x[1], reverse=True)[:15])
    ax.barh(list(top_parents.keys()), list(top_parents.values()), color='steelblue')
    ax.set_xlabel('Number of Subclasses')
    ax.set_title('LUCADA Ontology - Top Parent Classes by Subclass Count')
    plt.tight_layout()
    plt.show()
else:
    print("No parent hierarchy data to visualize.")

---
## 3. Guideline Rules Engine (NICE CG121)

Test the clinical guideline rules that drive treatment recommendations.

In [ ]:
from backend.src.ontology.guideline_rules import GuidelineRuleEngine, GuidelineRule

rule_engine = GuidelineRuleEngine()

print("=== NICE Lung Cancer Guideline Rules ===")
print(f"Total rules loaded: {len(rule_engine.NICE_GUIDELINES)}\n")

for rule in rule_engine.NICE_GUIDELINES:
    print(f"Rule {rule.rule_id}: {rule.name}")
    print(f"  Source: {rule.source}")
    print(f"  Treatment: {rule.recommended_treatment}")
    print(f"  Intent: {rule.treatment_intent}")
    print(f"  Evidence: {rule.evidence_level}")
    print(f"  Description: {rule.description[:100]}...")
    print()

In [ ]:
# Visualize guideline rule distribution
rules_df = pd.DataFrame([
    {
        'rule_id': r.rule_id,
        'treatment': r.recommended_treatment,
        'intent': r.treatment_intent,
        'evidence': r.evidence_level
    }
    for r in rule_engine.NICE_GUIDELINES
])

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Treatment types
rules_df['treatment'].value_counts().plot(kind='bar', ax=axes[0], color='teal')
axes[0].set_title('Rules by Treatment Type')
axes[0].tick_params(axis='x', rotation=45)

# Treatment intent
rules_df['intent'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.0f%%')
axes[1].set_title('Rules by Treatment Intent')

# Evidence level
rules_df['evidence'].value_counts().plot(kind='bar', ax=axes[2], color='coral')
axes[2].set_title('Rules by Evidence Level')
axes[2].tick_params(axis='x', rotation=45)

plt.suptitle('NICE CG121 Guideline Rules Distribution', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## 4. Individual Agent Testing

Test each of the 6 core agents independently.

In [ ]:
# Define test patients
test_patients = [
    {
        "name": "Jenny_Sesen",
        "sex": "F",
        "age": 72,
        "tnm_stage": "IIA",
        "histology_type": "Carcinosarcoma",
        "performance_status": 1,
        "fev1_percent": 75.0,
        "laterality": "Right",
        "comorbidities": [],
        "notes": "Canonical example from Sesen et al. paper"
    },
    {
        "name": "John_Smith",
        "sex": "M",
        "age": 65,
        "tnm_stage": "IB",
        "histology_type": "Adenocarcinoma",
        "performance_status": 0,
        "fev1_percent": 85.0,
        "laterality": "Left",
        "comorbidities": [],
        "notes": "Early stage NSCLC, ideal surgery candidate"
    },
    {
        "name": "Mary_Williams",
        "sex": "F",
        "age": 58,
        "tnm_stage": "IIIA",
        "histology_type": "SquamousCellCarcinoma",
        "performance_status": 1,
        "fev1_percent": 70.0,
        "laterality": "Right",
        "comorbidities": ["COPD"],
        "notes": "Locally advanced NSCLC with COPD"
    },
    {
        "name": "Robert_Johnson",
        "sex": "M",
        "age": 68,
        "tnm_stage": "IV",
        "histology_type": "Adenocarcinoma",
        "performance_status": 1,
        "fev1_percent": None,
        "laterality": "Right",
        "comorbidities": ["Hypertension"],
        "notes": "Metastatic NSCLC"
    },
    {
        "name": "Sarah_Davis",
        "sex": "F",
        "age": 62,
        "tnm_stage": "IIIB",
        "histology_type": "SmallCellCarcinoma",
        "performance_status": 2,
        "fev1_percent": 55.0,
        "laterality": "Left",
        "comorbidities": ["COPD", "Diabetes"],
        "notes": "SCLC with multiple comorbidities"
    }
]

print(f"Defined {len(test_patients)} test patients.")
for p in test_patients:
    print(f"  - {p['name']}: {p['histology_type']}, Stage {p['tnm_stage']}, PS {p['performance_status']}")

### 4.1 Agent 1: Ingestion Agent

In [ ]:
from backend.src.agents.ingestion_agent import IngestionAgent

ingestion_agent = IngestionAgent()

print("=== Ingestion Agent Results ===")
ingested_patients = []
for patient_data in test_patients:
    patient_fact, errors = ingestion_agent.execute(patient_data)
    if patient_fact:
        ingested_patients.append(patient_fact)
        print(f"\n[OK] {patient_fact.name}")
        print(f"  Patient ID: {patient_fact.patient_id}")
        print(f"  Stage: {patient_fact.tnm_stage}, Histology: {patient_fact.histology_type}")
        print(f"  PS: {patient_fact.performance_status}, FEV1: {patient_fact.fev1_percent}")
    else:
        print(f"\n[ERROR] {patient_data['name']}: {errors}")

print(f"\nSuccessfully ingested: {len(ingested_patients)}/{len(test_patients)}")

In [ ]:
# Test edge cases for ingestion
edge_cases = [
    {"name": "Invalid_Stage", "sex": "M", "age": 50, "tnm_stage": "Stage IIIA",
     "histology_type": "adenocarcinoma", "performance_status": 1},
    {"name": "Missing_Fields", "sex": "F", "age": 45},
    {"name": "Extreme_Age", "sex": "M", "age": 95, "tnm_stage": "IV",
     "histology_type": "SmallCellCarcinoma", "performance_status": 3},
]

print("=== Edge Case Testing ===")
for case in edge_cases:
    patient_fact, errors = ingestion_agent.execute(case)
    status = "OK" if patient_fact else "REJECTED"
    print(f"  [{status}] {case['name']}: errors={errors}")

### 4.2 Agent 2: Semantic Mapping Agent

In [ ]:
from backend.src.agents.semantic_mapping_agent import SemanticMappingAgent

semantic_agent = SemanticMappingAgent()

print("=== Semantic Mapping Results ===")
mapped_patients = []
for patient_fact in ingested_patients:
    patient_with_codes, confidence = semantic_agent.execute(patient_fact)
    mapped_patients.append(patient_with_codes)
    print(f"\n{patient_fact.name} (confidence: {confidence:.2f})")
    print(f"  SNOMED Diagnosis: {patient_with_codes.snomed_diagnosis_code}")
    print(f"  SNOMED Histology: {patient_with_codes.snomed_histology_code}")
    print(f"  SNOMED Stage: {patient_with_codes.snomed_stage_code}")
    print(f"  SNOMED PS: {patient_with_codes.snomed_ps_code}")
    print(f"  SNOMED Laterality: {patient_with_codes.snomed_laterality_code}")

### 4.3 Agent 3: Classification Agent

In [ ]:
from backend.src.agents.classification_agent import ClassificationAgent

classification_agent = ClassificationAgent()

print("=== Classification Results ===")
classifications = []
for patient in mapped_patients:
    result = classification_agent.execute(patient)
    classifications.append(result)
    print(f"\n{patient.name}:")
    print(f"  Scenario: {result.scenario}")
    print(f"  Confidence: {result.scenario_confidence:.2f}")
    print(f"  Reasoning: {result.reasoning_chain[:3]}")
    print(f"  Guidelines matched: {result.guideline_refs}")
    print(f"  Recommendations:")
    for rec in result.recommendations[:3]:
        print(f"    - {rec}")

### 4.4 Agent 4: Conflict Resolution Agent

In [ ]:
from backend.src.agents.conflict_resolution_agent import ConflictResolutionAgent

conflict_agent = ConflictResolutionAgent()

print("=== Conflict Resolution Results ===")
resolved_classifications = []
all_conflict_reports = []
for i, classification in enumerate(classifications):
    resolved, conflicts = conflict_agent.execute(classification)
    resolved_classifications.append(resolved)
    all_conflict_reports.append(conflicts)
    print(f"\n{mapped_patients[i].name}:")
    print(f"  Conflicts found: {len(conflicts)}")
    for c in conflicts:
        print(f"    - Type: {c.conflict_type}, Resolution: {c.resolution}")
    print(f"  Resolved scenario: {resolved.scenario}")
    print(f"  Final recommendations: {len(resolved.recommendations)}")

### 4.5 Agent 6: Explanation Agent

Note: The ExplanationAgent generates MDT summaries. If Ollama is not available, it falls back to template-based generation.

In [ ]:
from backend.src.agents.explanation_agent import ExplanationAgent

explanation_agent = ExplanationAgent()

print("=== MDT Summary Generation ===")
mdt_summaries = []
for i, (patient, classification) in enumerate(zip(mapped_patients, resolved_classifications)):
    try:
        summary = explanation_agent.execute(
            patient_fact=patient,
            classification=classification
        )
        mdt_summaries.append(summary)
        print(f"\n{'='*60}")
        print(f"MDT Summary for {patient.name}")
        print(f"{'='*60}")
        print(f"Clinical Summary: {summary.clinical_summary[:200]}...")
        print(f"Scenario: {summary.classification_scenario}")
        print(f"Confidence: {summary.scenario_confidence:.2f}")
        print(f"Key Considerations: {summary.key_considerations[:3]}")
        print(f"Discussion Points: {summary.discussion_points[:3]}")
    except Exception as e:
        print(f"\n[WARN] {patient.name}: Explanation generation failed - {e}")
        mdt_summaries.append(None)

---
## 5. Full 6-Agent Workflow (LangGraph Pipeline)

Run the complete end-to-end workflow through all 6 agents.

In [ ]:
from backend.src.agents.lca_workflow import LCAWorkflow

# Initialize workflow without Neo4j persistence (for standalone testing)
workflow = LCAWorkflow(persist_results=False)

print("=== Full 6-Agent Workflow Execution ===")
workflow_results = []

for patient_data in test_patients:
    print(f"\nProcessing: {patient_data['name']}")
    start_time = time.time()
    
    try:
        result = workflow.run(patient_data)
        elapsed = time.time() - start_time
        workflow_results.append({
            'name': patient_data['name'],
            'result': result,
            'time_seconds': elapsed,
            'success': True
        })
        
        print(f"  Status: {result.get('workflow_status', 'unknown')}")
        print(f"  Agent Chain: {result.get('agent_chain', [])}")
        print(f"  Time: {elapsed:.2f}s")
        
        if result.get('classification'):
            cls = result['classification']
            scenario = cls.scenario if hasattr(cls, 'scenario') else str(cls)
            print(f"  Scenario: {scenario}")
        
        if result.get('mdt_summary'):
            summary = result['mdt_summary']
            text = summary.clinical_summary if hasattr(summary, 'clinical_summary') else str(summary)
            print(f"  MDT Summary: {text[:150]}...")
    except Exception as e:
        elapsed = time.time() - start_time
        workflow_results.append({
            'name': patient_data['name'],
            'result': None,
            'time_seconds': elapsed,
            'success': False,
            'error': str(e)
        })
        print(f"  [ERROR] {e}")

# Summary
successes = sum(1 for r in workflow_results if r['success'])
print(f"\n=== Workflow Summary ===")
print(f"Total: {len(workflow_results)}, Success: {successes}, Failed: {len(workflow_results) - successes}")
print(f"Avg time: {np.mean([r['time_seconds'] for r in workflow_results]):.2f}s")

In [ ]:
# Visualize workflow execution times
fig, ax = plt.subplots(figsize=(10, 5))
names = [r['name'] for r in workflow_results]
times = [r['time_seconds'] for r in workflow_results]
colors = ['green' if r['success'] else 'red' for r in workflow_results]

ax.barh(names, times, color=colors)
ax.set_xlabel('Execution Time (seconds)')
ax.set_title('6-Agent Workflow Execution Time per Patient')
for i, v in enumerate(times):
    ax.text(v + 0.01, i, f'{v:.2f}s', va='center')
plt.tight_layout()
plt.show()

---
## 6. Specialized Agents

Test the domain-specific specialized agents.

In [ ]:
from backend.src.agents.biomarker_agent import BiomarkerAgent, BiomarkerProfile

biomarker_agent = BiomarkerAgent()

# Test with different biomarker profiles
biomarker_profiles = [
    BiomarkerProfile(egfr_mutation="positive", egfr_mutation_type="Ex19del", pdl1_tps=80.0),
    BiomarkerProfile(alk_rearrangement="positive", pdl1_tps=30.0),
    BiomarkerProfile(pdl1_tps=90.0, tmb_score=15.0),
    BiomarkerProfile(braf_mutation="positive"),
    BiomarkerProfile(),  # No actionable mutations
]

print("=== Biomarker Agent Results ===")
# Use the advanced NSCLC patient for biomarker testing
if len(mapped_patients) >= 4:
    test_patient = mapped_patients[3]  # Robert_Johnson, Stage IV Adenocarcinoma
    for i, profile in enumerate(biomarker_profiles):
        proposal = biomarker_agent.execute(test_patient, biomarker_profile=profile)
        print(f"\nProfile {i+1}: {profile}")
        print(f"  Treatment: {proposal.treatment}")
        print(f"  Confidence: {proposal.confidence:.2f}")
        print(f"  Rationale: {proposal.rationale[:150]}")

In [ ]:
from backend.src.agents.nsclc_agent import NSCLCAgent

nsclc_agent = NSCLCAgent()

print("=== NSCLC Agent - Stage-Specific Pathways ===")
for patient in mapped_patients:
    # Skip SCLC patients
    if patient.histology_type in ['SmallCellCarcinoma']:
        continue
    try:
        proposal = nsclc_agent.execute(patient)
        print(f"\n{patient.name} (Stage {patient.tnm_stage}, {patient.histology_type}):")
        print(f"  Treatment: {proposal.treatment}")
        print(f"  Confidence: {proposal.confidence:.2f}")
        print(f"  Rationale: {proposal.rationale[:200]}")
    except Exception as e:
        print(f"\n{patient.name}: Error - {e}")

In [ ]:
from backend.src.agents.comorbidity_agent import ComorbidityAgent

comorbidity_agent = ComorbidityAgent()

print("=== Comorbidity Agent Results ===")
for patient in mapped_patients:
    try:
        proposal = comorbidity_agent.execute(patient)
        print(f"\n{patient.name} (comorbidities: {patient.comorbidities}):")
        print(f"  Risk Assessment: {proposal.treatment}")
        print(f"  Confidence: {proposal.confidence:.2f}")
        print(f"  Rationale: {proposal.rationale[:200]}")
    except Exception as e:
        print(f"\n{patient.name}: Error - {e}")

---
## 7. Multi-Patient Cohort Processing

Process a larger cohort from the sample patients and analyze distribution.

In [ ]:
from data.sample_patients import SAMPLE_PATIENTS

print(f"Sample patients available: {len(SAMPLE_PATIENTS)}")
for p in SAMPLE_PATIENTS:
    print(f"  - {p['name']}: Stage {p['tnm_stage']}, {p['histology_type']}, PS {p['performance_status']}")

In [ ]:
# Process all sample patients through the ingestion + classification pipeline
cohort_results = []

for patient_data in SAMPLE_PATIENTS:
    try:
        # Ingestion
        patient_fact, errors = ingestion_agent.execute(patient_data)
        if not patient_fact:
            continue
        
        # Semantic mapping
        patient_with_codes, confidence = semantic_agent.execute(patient_fact)
        
        # Classification
        classification = classification_agent.execute(patient_with_codes)
        
        cohort_results.append({
            'name': patient_data['name'],
            'stage': patient_data['tnm_stage'],
            'histology': patient_data['histology_type'],
            'ps': patient_data['performance_status'],
            'age': patient_data['age'],
            'scenario': classification.scenario,
            'confidence': classification.scenario_confidence,
            'num_recommendations': len(classification.recommendations),
            'guidelines': classification.guideline_refs,
            'mapping_confidence': confidence
        })
    except Exception as e:
        print(f"Error processing {patient_data.get('name', 'unknown')}: {e}")

cohort_df = pd.DataFrame(cohort_results)
print(f"\nCohort processed: {len(cohort_df)} patients")
cohort_df

In [ ]:
# Cohort analysis visualizations
if len(cohort_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Stage distribution
    cohort_df['stage'].value_counts().plot(kind='bar', ax=axes[0, 0], color='steelblue')
    axes[0, 0].set_title('Stage Distribution')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Histology distribution
    cohort_df['histology'].value_counts().plot(kind='bar', ax=axes[0, 1], color='teal')
    axes[0, 1].set_title('Histology Distribution')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Classification confidence
    axes[1, 0].hist(cohort_df['confidence'], bins=10, color='coral', edgecolor='black')
    axes[1, 0].set_title('Classification Confidence Distribution')
    axes[1, 0].set_xlabel('Confidence')
    
    # Age vs Performance Status
    scatter = axes[1, 1].scatter(
        cohort_df['age'], cohort_df['ps'],
        c=cohort_df['confidence'], cmap='RdYlGn', s=100, edgecolors='black'
    )
    axes[1, 1].set_xlabel('Age')
    axes[1, 1].set_ylabel('Performance Status')
    axes[1, 1].set_title('Age vs PS (color = confidence)')
    plt.colorbar(scatter, ax=axes[1, 1], label='Confidence')
    
    plt.suptitle('Cohort Analysis', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No cohort data to visualize.")

---
## 8. Synthetic Data Generation & Batch Analysis

Generate a larger synthetic cohort and run batch processing.

In [ ]:
from data.synthetic_patient_generator import SyntheticPatientGenerator

generator = SyntheticPatientGenerator()

# Generate 50 synthetic patients
N_SYNTHETIC = 50
synthetic_patients = [generator.generate_patient() for _ in range(N_SYNTHETIC)]

print(f"Generated {len(synthetic_patients)} synthetic patients")
print(f"\nStage distribution:")
stage_counts = {}
for p in synthetic_patients:
    stage_counts[p.tnm_stage] = stage_counts.get(p.tnm_stage, 0) + 1
for stage, count in sorted(stage_counts.items()):
    print(f"  {stage}: {count} ({count/len(synthetic_patients)*100:.0f}%)")

In [ ]:
# Batch process synthetic patients through the pipeline
batch_results = []
batch_errors = []

print(f"Processing {N_SYNTHETIC} synthetic patients...")
start_time = time.time()

for i, patient in enumerate(synthetic_patients):
    try:
        patient_data = patient.to_dict()
        patient_data['name'] = patient.name  # ensure name field
        
        # Ingestion
        patient_fact, errors = ingestion_agent.execute(patient_data)
        if not patient_fact:
            batch_errors.append({'patient': patient.name, 'stage': 'ingestion', 'error': str(errors)})
            continue
        
        # Semantic mapping
        patient_with_codes, confidence = semantic_agent.execute(patient_fact)
        
        # Classification
        classification = classification_agent.execute(patient_with_codes)
        
        # Conflict resolution
        resolved, conflicts = conflict_agent.execute(classification)
        
        batch_results.append({
            'name': patient.name,
            'age': patient.age_at_diagnosis,
            'sex': patient.sex,
            'stage': patient.tnm_stage,
            'histology': patient.histology_type,
            'ps': patient.performance_status,
            'fev1': patient.fev1_percent,
            'comorbidities': len(patient.comorbidities or []),
            'scenario': resolved.scenario,
            'confidence': resolved.scenario_confidence,
            'num_recommendations': len(resolved.recommendations),
            'num_conflicts': len(conflicts),
            'mapping_confidence': confidence
        })
        
        if (i + 1) % 10 == 0:
            print(f"  Processed {i+1}/{N_SYNTHETIC}...")
            
    except Exception as e:
        batch_errors.append({'patient': patient.name, 'stage': 'pipeline', 'error': str(e)})

total_time = time.time() - start_time
batch_df = pd.DataFrame(batch_results)

print(f"\n=== Batch Processing Complete ===")
print(f"Total time: {total_time:.2f}s")
print(f"Success: {len(batch_results)}, Errors: {len(batch_errors)}")
print(f"Throughput: {len(batch_results)/total_time:.1f} patients/sec")

In [ ]:
# Comprehensive batch analysis
if len(batch_df) > 0:
    print("=== Batch Statistics ===")
    print(f"\nConfidence: mean={batch_df['confidence'].mean():.3f}, "
          f"std={batch_df['confidence'].std():.3f}, "
          f"min={batch_df['confidence'].min():.3f}, "
          f"max={batch_df['confidence'].max():.3f}")
    print(f"\nMapping confidence: mean={batch_df['mapping_confidence'].mean():.3f}")
    print(f"\nRecommendations per patient: mean={batch_df['num_recommendations'].mean():.1f}")
    print(f"Conflicts per patient: mean={batch_df['num_conflicts'].mean():.1f}")
    
    print(f"\nScenario distribution:")
    for scenario, count in batch_df['scenario'].value_counts().items():
        print(f"  {scenario}: {count} ({count/len(batch_df)*100:.0f}%)")

In [ ]:
# Batch visualizations
if len(batch_df) > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # 1. Stage vs Confidence
    batch_df.boxplot(column='confidence', by='stage', ax=axes[0, 0])
    axes[0, 0].set_title('Classification Confidence by Stage')
    axes[0, 0].set_xlabel('TNM Stage')
    
    # 2. Histology distribution
    batch_df['histology'].value_counts().plot(kind='pie', ax=axes[0, 1], autopct='%1.0f%%')
    axes[0, 1].set_title('Histology Distribution')
    
    # 3. Age distribution by stage
    for stage in batch_df['stage'].unique():
        subset = batch_df[batch_df['stage'] == stage]
        axes[0, 2].hist(subset['age'], alpha=0.5, label=stage, bins=10)
    axes[0, 2].set_title('Age Distribution by Stage')
    axes[0, 2].set_xlabel('Age')
    axes[0, 2].legend(fontsize=8)
    
    # 4. Recommendations heatmap: stage x histology
    pivot = batch_df.pivot_table(
        values='num_recommendations', index='stage', columns='histology', aggfunc='mean'
    )
    sns.heatmap(pivot, annot=True, fmt='.1f', cmap='YlOrRd', ax=axes[1, 0])
    axes[1, 0].set_title('Avg Recommendations (Stage x Histology)')
    
    # 5. Confidence vs number of comorbidities
    axes[1, 1].scatter(batch_df['comorbidities'], batch_df['confidence'],
                       c=batch_df['ps'], cmap='viridis', s=60, edgecolors='black')
    axes[1, 1].set_xlabel('Number of Comorbidities')
    axes[1, 1].set_ylabel('Classification Confidence')
    axes[1, 1].set_title('Confidence vs Comorbidities (color=PS)')
    
    # 6. Performance status vs conflicts
    batch_df.boxplot(column='num_conflicts', by='ps', ax=axes[1, 2])
    axes[1, 2].set_title('Conflicts by Performance Status')
    axes[1, 2].set_xlabel('Performance Status (WHO)')
    
    plt.suptitle(f'Synthetic Cohort Analysis (N={len(batch_df)})', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

---
## 9. Analytics Modules

Test survival analysis, uncertainty quantification, and counterfactual reasoning.

### 9.1 Survival Analysis

In [ ]:
from backend.src.analytics.survival_analyzer import SurvivalAnalyzer

survival_analyzer = SurvivalAnalyzer()

# Generate synthetic survival data for demonstration
np.random.seed(42)
n_patients = 100

survival_data = []
for _ in range(n_patients):
    stage = np.random.choice(['IA', 'IB', 'IIA', 'IIB', 'IIIA', 'IIIB', 'IV'],
                             p=[0.10, 0.08, 0.12, 0.10, 0.15, 0.15, 0.30])
    treatment = np.random.choice(['Surgery', 'Chemotherapy', 'Chemoradiotherapy', 'Immunotherapy'])
    
    # Simulate survival based on stage
    base_survival = {'IA': 1800, 'IB': 1500, 'IIA': 1200, 'IIB': 1000,
                     'IIIA': 700, 'IIIB': 500, 'IV': 300}
    survival_days = max(30, int(np.random.exponential(base_survival[stage])))
    event = 1 if np.random.random() < 0.7 else 0  # 70% observed events
    
    survival_data.append({
        'patient_id': f'SYN-{_:03d}',
        'stage': stage,
        'treatment': treatment,
        'survival_days': survival_days,
        'event': event
    })

survival_df = pd.DataFrame(survival_data)
print(f"Survival dataset: {len(survival_df)} patients")
print(f"\nMedian survival by stage:")
print(survival_df.groupby('stage')['survival_days'].median().sort_values(ascending=False))

In [ ]:
# Kaplan-Meier survival curves by stage
try:
    from lifelines import KaplanMeierFitter
    from lifelines.statistics import logrank_test
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # By stage (grouped for clarity)
    kmf = KaplanMeierFitter()
    stage_groups = {'Early (I-II)': ['IA', 'IB', 'IIA', 'IIB'],
                    'Locally Advanced (III)': ['IIIA', 'IIIB'],
                    'Metastatic (IV)': ['IV']}
    
    for label, stages in stage_groups.items():
        mask = survival_df['stage'].isin(stages)
        kmf.fit(survival_df.loc[mask, 'survival_days'],
                event_observed=survival_df.loc[mask, 'event'],
                label=label)
        kmf.plot_survival_function(ax=axes[0])
    
    axes[0].set_title('Kaplan-Meier Survival by Stage Group')
    axes[0].set_xlabel('Days')
    axes[0].set_ylabel('Survival Probability')
    
    # By treatment
    for treatment in survival_df['treatment'].unique():
        mask = survival_df['treatment'] == treatment
        kmf.fit(survival_df.loc[mask, 'survival_days'],
                event_observed=survival_df.loc[mask, 'event'],
                label=treatment)
        kmf.plot_survival_function(ax=axes[1])
    
    axes[1].set_title('Kaplan-Meier Survival by Treatment')
    axes[1].set_xlabel('Days')
    axes[1].set_ylabel('Survival Probability')
    
    plt.tight_layout()
    plt.show()
    
    # Log-rank test: early vs metastatic
    early = survival_df[survival_df['stage'].isin(['IA', 'IB', 'IIA', 'IIB'])]
    metastatic = survival_df[survival_df['stage'] == 'IV']
    result = logrank_test(
        early['survival_days'], metastatic['survival_days'],
        event_observed_A=early['event'], event_observed_B=metastatic['event']
    )
    print(f"Log-rank test (Early vs Metastatic): p={result.p_value:.4f}")
    
except ImportError:
    print("lifelines not installed. Showing basic survival statistics instead.")
    print(survival_df.groupby('stage')['survival_days'].describe())

### 9.2 Uncertainty Quantification

In [ ]:
from backend.src.analytics.uncertainty_quantifier import UncertaintyQuantifier
from backend.src.db.models import TreatmentRecommendation, EvidenceLevel, TreatmentIntent

uq = UncertaintyQuantifier()

print("=== Uncertainty Quantification ===")

# Create sample recommendations for uncertainty analysis
sample_recommendations = [
    TreatmentRecommendation(
        patient_id="TEST-001",
        primary_treatment="Surgery (Lobectomy)",
        treatment_intent=TreatmentIntent.CURATIVE,
        evidence_level=EvidenceLevel.GRADE_A,
        confidence_score=0.92,
        rationale="Early stage NSCLC with good PS - standard surgical approach",
        guideline_references=["NICE CG121 R2"]
    ),
    TreatmentRecommendation(
        patient_id="TEST-002",
        primary_treatment="Concurrent Chemoradiotherapy",
        treatment_intent=TreatmentIntent.CURATIVE,
        evidence_level=EvidenceLevel.GRADE_B,
        confidence_score=0.75,
        rationale="Locally advanced NSCLC with comorbidities",
        guideline_references=["NICE CG121 R6"]
    ),
    TreatmentRecommendation(
        patient_id="TEST-003",
        primary_treatment="Palliative Chemotherapy",
        treatment_intent=TreatmentIntent.PALLIATIVE,
        evidence_level=EvidenceLevel.GRADE_A,
        confidence_score=0.85,
        rationale="Metastatic NSCLC, PS 1",
        guideline_references=["NICE CG121 R1"]
    ),
]

uncertainty_results = []
for i, (rec, patient) in enumerate(zip(sample_recommendations, ingested_patients[:3])):
    try:
        metrics = uq.quantify_recommendation_uncertainty(rec, patient)
        uncertainty_results.append(metrics)
        print(f"\n{patient.name} - {rec.primary_treatment}:")
        print(f"  Confidence: {metrics.confidence_score:.3f}")
        print(f"  Epistemic uncertainty: {metrics.epistemic_uncertainty:.3f}")
        print(f"  Aleatoric uncertainty: {metrics.aleatoric_uncertainty:.3f}")
        print(f"  Total uncertainty: {metrics.total_uncertainty:.3f}")
        print(f"  Confidence level: {metrics.confidence_level}")
        print(f"  Explanation: {metrics.explanation[:150]}")
    except Exception as e:
        print(f"\nError for {patient.name}: {e}")

In [ ]:
# Visualize uncertainty
if uncertainty_results:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    names = [ingested_patients[i].name for i in range(len(uncertainty_results))]
    epistemic = [m.epistemic_uncertainty for m in uncertainty_results]
    aleatoric = [m.aleatoric_uncertainty for m in uncertainty_results]
    
    # Stacked bar: epistemic vs aleatoric
    x = np.arange(len(names))
    axes[0].bar(x, epistemic, label='Epistemic', color='steelblue')
    axes[0].bar(x, aleatoric, bottom=epistemic, label='Aleatoric', color='coral')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(names, rotation=45)
    axes[0].set_ylabel('Uncertainty')
    axes[0].set_title('Uncertainty Decomposition')
    axes[0].legend()
    
    # Confidence with error bars
    confidences = [m.confidence_score for m in uncertainty_results]
    totals = [m.total_uncertainty for m in uncertainty_results]
    axes[1].bar(x, confidences, yerr=totals, capsize=5, color='teal', alpha=0.7)
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(names, rotation=45)
    axes[1].set_ylabel('Confidence Score')
    axes[1].set_title('Recommendation Confidence with Uncertainty Bounds')
    axes[1].set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.show()

### 9.3 Counterfactual Analysis

In [ ]:
from backend.src.analytics.counterfactual_engine import CounterfactualEngine

cf_engine = CounterfactualEngine(workflow=workflow)

print("=== Counterfactual Analysis ===")
print("What-if scenarios for treatment decision sensitivity\n")

# Analyze counterfactuals for the canonical Jenny_Sesen case
jenny_data = test_patients[0]

try:
    cf_result = cf_engine.analyze_counterfactuals(
        patient_data=jenny_data,
        attributes_to_vary=['tnm_stage', 'performance_status', 'fev1_percent']
    )
    
    print(f"Patient: {cf_result.patient_id}")
    print(f"Original recommendation: {cf_result.original_recommendation}")
    print(f"\nCounterfactual scenarios:")
    for cf in cf_result.counterfactuals:
        print(f"  - {cf.get('description', cf)}")
    print(f"\nActionable interventions: {cf_result.actionable_interventions}")
    print(f"Sensitivity: {cf_result.sensitivity_analysis}")
except Exception as e:
    print(f"Counterfactual analysis error: {e}")
    print("\nRunning manual what-if analysis instead...")
    
    # Manual what-if: vary stage
    print("\nWhat-if: Stage variation for Jenny_Sesen")
    stages = ['IA', 'IIA', 'IIIA', 'IV']
    for stage in stages:
        modified = jenny_data.copy()
        modified['tnm_stage'] = stage
        pf, _ = ingestion_agent.execute(modified)
        if pf:
            pwc, _ = semantic_agent.execute(pf)
            cls = classification_agent.execute(pwc)
            print(f"  Stage {stage}: {cls.scenario} (confidence: {cls.scenario_confidence:.2f})")

---
## 10. Visualization & Reporting

Generate final summary visualizations and a consolidated report.

In [ ]:
# Treatment recommendation decision matrix
if len(batch_df) > 0:
    # Cross-tabulation: stage x scenario
    ct = pd.crosstab(batch_df['stage'], batch_df['scenario'])
    
    fig, ax = plt.subplots(figsize=(14, 6))
    ct.plot(kind='bar', stacked=True, ax=ax, colormap='Set3')
    ax.set_title('Treatment Scenarios by TNM Stage')
    ax.set_xlabel('TNM Stage')
    ax.set_ylabel('Count')
    ax.legend(title='Scenario', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
    plt.tight_layout()
    plt.show()

In [ ]:
# Comprehensive performance summary
print("=" * 70)
print("EXPERIMENT SUMMARY REPORT")
print("=" * 70)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n--- Ontology ---")
print(f"  Classes: {len(list(onto.classes()))}")
print(f"  Object Properties: {len(list(onto.object_properties()))}")
print(f"  Data Properties: {len(list(onto.data_properties()))}")
print(f"  SNOMED Codes: {len(snomed.LUNG_CANCER_CODES)}")
print(f"  Guideline Rules: {len(rule_engine.NICE_GUIDELINES)}")

print(f"\n--- Agent Pipeline ---")
print(f"  Test patients processed: {len(test_patients)}")
print(f"  Workflow executions: {len(workflow_results)}")
print(f"  Workflow success rate: {successes}/{len(workflow_results)}")
if workflow_results:
    print(f"  Avg workflow time: {np.mean([r['time_seconds'] for r in workflow_results]):.2f}s")

print(f"\n--- Batch Processing ---")
print(f"  Synthetic patients: {N_SYNTHETIC}")
print(f"  Successfully processed: {len(batch_results)}")
print(f"  Errors: {len(batch_errors)}")
if len(batch_df) > 0:
    print(f"  Mean confidence: {batch_df['confidence'].mean():.3f}")
    print(f"  Mean mapping confidence: {batch_df['mapping_confidence'].mean():.3f}")
    print(f"  Throughput: {len(batch_results)/total_time:.1f} patients/sec")

print(f"\n--- Analytics ---")
print(f"  Survival dataset: {len(survival_df)} patients")
print(f"  Uncertainty quantified: {len(uncertainty_results)} recommendations")

print("\n" + "=" * 70)
print("All experiments completed.")
print("=" * 70)

In [ ]:
# Export results to CSV for external analysis
if len(batch_df) > 0:
    output_dir = os.path.join(PROJECT_ROOT, 'experiment_outputs')
    os.makedirs(output_dir, exist_ok=True)
    
    batch_df.to_csv(os.path.join(output_dir, 'batch_results.csv'), index=False)
    survival_df.to_csv(os.path.join(output_dir, 'survival_data.csv'), index=False)
    
    if len(cohort_df) > 0:
        cohort_df.to_csv(os.path.join(output_dir, 'cohort_results.csv'), index=False)
    
    print(f"Results exported to: {output_dir}/")
    print(f"  - batch_results.csv ({len(batch_df)} rows)")
    print(f"  - survival_data.csv ({len(survival_df)} rows)")
    if len(cohort_df) > 0:
        print(f"  - cohort_results.csv ({len(cohort_df)} rows)")
else:
    print("No batch data to export.")